In [1]:
import requests
from bs4 import BeautifulSoup

html = requests.get('https://www.bestattungen.de/verzeichnis.html').text

soup = BeautifulSoup(html)

linkes = []

possible_links = soup.find_all('a')

for link in possible_links:
    if link.has_attr('href'):
        linkes.append(link.attrs['href'])
#        print (link.attrs['href'])

In [2]:
from pandas import DataFrame

df_link = DataFrame (linkes, columns = ['path'])
df_link = df_link.drop_duplicates()
df_link.shape

(36, 1)

In [3]:
url_top = 'https://www.bestattungen.de'

df_combine = df_link[df_link['path'].str.contains('/verzeichnis/bestatter')]
df_combine = df_combine.drop(df_combine.index[-1])
df_combine['domain'] = url_top
df_combine['url'] = df_combine['domain'] + df_combine['path']
df_combine.shape

(16, 3)

In [4]:
df_combine

,path,domain,url
18,/verzeichnis/bestatter-in-baden-wuerttemberg.html,https://www.bestattungen.de,https://www.bestattungen.de/verzeichnis/bestat...
19,/verzeichnis/bestatter-in-bayern.html,https://www.bestattungen.de,https://www.bestattungen.de/verzeichnis/bestat...
20,/verzeichnis/bestatter-in-berlin.html,https://www.bestattungen.de,https://www.bestattungen.de/verzeichnis/bestat...
21,/verzeichnis/bestatter-in-brandenburg.html,https://www.bestattungen.de,https://www.bestattungen.de/verzeichnis/bestat...
22,/verzeichnis/bestatter-in-bremen.html,https://www.bestattungen.de,https://www.bestattungen.de/verzeichnis/bestat...
23,/verzeichnis/bestatter-in-hamburg.html,https://www.bestattungen.de,https://www.bestattungen.de/verzeichnis/bestat...
24,/verzeichnis/bestatter-in-hessen.html,https://www.bestattungen.de,https://www.bestattungen.de/verzeichnis/bestat...
25,/verzeichnis/bestatter-in-mecklenburg-vorpomme...,https://www.bestattungen.de,https://www.bestattungen.de/verzeichnis/bestat...
26,/verzeichnis/bestatter-in-niedersachsen.html,https://www.bestattungen.de,https://www.bestattungen.de/verzeichnis/bestat...
27,/verzeichnis/bestatter-in-nordrhein-westfalen....,https://www.bestattungen.de,https://www.bestattungen.de/verzeichnis/bestat...


In [5]:
import shutil
from os import makedirs

bundesland_local = 'bestattungen/bundesland/'

makedirs(bundesland_local, exist_ok = True)
shutil.rmtree(bundesland_local)
makedirs(bundesland_local, exist_ok = True)

In [6]:
for index, row in df_combine.iterrows():
    
    url = row['url']
#    print('Downloading :', url)
    resp = requests.get(url)
    
    fname = bundesland_local + str(index) + '.html'
#    print('Saving: ', fname)
    
    with open(fname, 'w') as wf:
        wf.write(resp.text)

print('\n')
print('done...')



done...


In [7]:
from glob import glob
from os.path import join

html_filenames = glob(join(bundesland_local, '*.html'))

In [8]:
region_local = 'bestattungen/region/'

makedirs(region_local, exist_ok = True)
shutil.rmtree(region_local)
makedirs(region_local, exist_ok = True)

In [9]:
from urllib.parse import urljoin

for hname in html_filenames:
    
    with open(hname, 'r') as rf:
        txt = rf.read()
        soup = BeautifulSoup(txt, 'lxml')
        
        for active in soup.find_all('div', {'aside-wrapper'}):
            active_link = active.find_all('li', {'parent active openMenu current'})
#            print (active_link)
            
            for each in active_link:
                url = each.find_all('li')
#                print('Downloading: ', url)
                
                for link in url:
                    href = link.find('a').attrs['href']
                    url = urljoin(url_top, href)
#                    print('Downloading: ', url)
                    resp = requests.get(url)
                    
                    
                    fn = href.replace('/', '-').strip('-')
                    full_fname = join(region_local, fn)
#                    print('Saving ', full_fname)
                    with open(full_fname, 'w') as wf:
                        wf.write(resp.text)

print('\n')
print('done...')



done...


In [10]:
import re

regex = re.compile(r'[\n\r\t]')

import pandas as pd
import glob
import os.path

df_bestattungen = pd.DataFrame()

count = 0

for file_name in glob.glob(os.path.join(region_local, '*.html')):
    with open(file_name) as html_file:
        soup = BeautifulSoup(html_file)
#        print (file_name)
        
        for i in soup.find('div', {'page'}):
            company_all = soup.find_all('div')
#            print('Number of cards found       : ', len(company_all))
            count += len(company_all)
#            print('Total Number of cards found : ', count)
#            print('\n')
#            print(company_all)

            for select in company_all:
                company = getattr(select.find('h2'), 'text', None)
#                print(company)

                address = getattr(select.find('div', {'address'}), 'text', None)
#                print (address)

                df_bestattungen = df_bestattungen.append(pd.DataFrame([[company, address]],
                                                                      columns=['Marke', 'Adresse']))

print('\n')
print('Total Number of cards found : ', count)
print('done...')



Total Number of cards found :  409402
done...


In [11]:
df_bestattungen.shape

(409402, 2)

In [12]:
df_bestattungen.reset_index(drop = True, inplace = True)
#df_bestattungen.head(25)

In [13]:
df_bestattungen = df_bestattungen[df_bestattungen['Marke'].notna()]
df_bestattungen = df_bestattungen[df_bestattungen['Adresse'].notna()]
df_bestattungen = df_bestattungen.drop_duplicates('Marke', keep='first')
#df_bestattungen.head(25)

In [14]:
df_bestattungen[['Strasse']] = df_bestattungen.Adresse.str.extract('(.+)', expand = True)
df_bestattungen[['Plz', 'Ort']] = df_bestattungen.Adresse.str.extract('([0-9]{5})[ ](.+)', expand = True)
#df_bestattungen.head(25)

In [15]:
df_bestattungen.isnull().sum(axis = 0)

Marke      0
Adresse    0
Strasse    0
Plz        2
Ort        2
dtype: int64

In [16]:
df_bestattungen = df_bestattungen[df_bestattungen['Plz'].notna()]

In [17]:
df_bestattungen['Plz'] = df_bestattungen['Plz'].astype(int)
df_bestattungen['Plz'] = ["%05d" % n for n in df_bestattungen['Plz']]
#df_bestattungen.head(25)

In [18]:
df_bestattungen['Source'] = 'https://www.bestattungen.de/verzeichnis.html'
#df_bestattungen.head(25)

In [19]:
df_bestattungen = df_bestattungen.drop('Adresse', axis = 1)
#df_bestattungen.head(25)

In [20]:
df_bestattungen['Marke'] = df_bestattungen['Marke'].str.replace('\n',' ')
#df_bestattungen.head(25)

In [21]:
df_bestattungen

,Marke,Strasse,Plz,Ort,Source
11,Bestattungen Horn,Neuhauser Straße 93,78532,Tuttlingen,https://www.bestattungen.de/verzeichnis.html
29,Bestattungen Merkt,Hauptstr. 136,78549,Spaichingen,https://www.bestattungen.de/verzeichnis.html
36,Bestattungsinstitut Ella Sichler e. K.,Brunnentalstr. 1,78532,Tuttlingen,https://www.bestattungen.de/verzeichnis.html
42,Bestattungsinstitut Neipp e.k.,Göldenbühlstraße 15,78647,Trossingen,https://www.bestattungen.de/verzeichnis.html
49,Franz Fetzer Bestattungen,Hauptstraße 142,78588,Denkingen,https://www.bestattungen.de/verzeichnis.html
...,...,...,...,...,...
408203,Hans-Jürgen und Renate Müller Bestattungsin...,Valentina-Teresckowa-Str. 9,00644,Aschersleben,https://www.bestattungen.de/verzeichnis.html
408210,Heinze Bestattungen,Arnstedtstraße 91,39240,Calbe (Saale),https://www.bestattungen.de/verzeichnis.html
408216,Ingolf Heiduk Bestattungsinstitut,Dr.-M.Luther-Str. 3,39218,Schönebeck,https://www.bestattungen.de/verzeichnis.html
408223,U. Pröseler Bestattungen,Damaschkestraße 7,39240,Zuchau,https://www.bestattungen.de/verzeichnis.html


In [22]:
df_bestattungen.to_csv('bestattungen/df_bestattungen_20200609.csv', sep = ';', index = False)

In [23]:
import pandas as pd

df = pd.read_csv('bestattungen/df_bestattungen_20200609.csv', sep = ';')
df

,Marke,Strasse,Plz,Ort,Source
0,Bestattungen Horn,Neuhauser Straße 93,78532,Tuttlingen,https://www.bestattungen.de/verzeichnis.html
1,Bestattungen Merkt,Hauptstr. 136,78549,Spaichingen,https://www.bestattungen.de/verzeichnis.html
2,Bestattungsinstitut Ella Sichler e. K.,Brunnentalstr. 1,78532,Tuttlingen,https://www.bestattungen.de/verzeichnis.html
3,Bestattungsinstitut Neipp e.k.,Göldenbühlstraße 15,78647,Trossingen,https://www.bestattungen.de/verzeichnis.html
4,Franz Fetzer Bestattungen,Hauptstraße 142,78588,Denkingen,https://www.bestattungen.de/verzeichnis.html
...,...,...,...,...,...
5756,Hans-Jürgen und Renate Müller Bestattungsin...,Valentina-Teresckowa-Str. 9,644,Aschersleben,https://www.bestattungen.de/verzeichnis.html
5757,Heinze Bestattungen,Arnstedtstraße 91,39240,Calbe (Saale),https://www.bestattungen.de/verzeichnis.html
5758,Ingolf Heiduk Bestattungsinstitut,Dr.-M.Luther-Str. 3,39218,Schönebeck,https://www.bestattungen.de/verzeichnis.html
5759,U. Pröseler Bestattungen,Damaschkestraße 7,39240,Zuchau,https://www.bestattungen.de/verzeichnis.html
